In [115]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
import math

In [2]:
import sys
import os
import glob
import hdf5_getters
import re

In [3]:
# build path dictionary
path_dict_song = {} 
basedir = "../MillionSongSubset/data/" 
ext = ".h5"
for root, dirs, files in os.walk(basedir):
    for f in files:
        path_track = os.path.join(root,f)
        track_ID = f.split('.')[0]
        path_dict_song[track_ID] = path_track

In [4]:
# build song to track dic:
count = 0
song_track_dic = {}
with open('taste_profile_song_to_tracks.txt') as f:
    for line in f:
#         print(line)
        count += 1
        if count < 0:
            break
        else: 
            valTuple = line.split('\t')
            if len(valTuple) > 1:
                songID = valTuple[0]
                trackID = valTuple[1].split('\n')[0]
                song_track_dic[songID] = trackID
# We have total 3 million song track data

In [5]:
songCSV = pd.read_csv('SongCSV.csv')

In [6]:
# userID = 'b80344d063b5ccb3212f76538f3d9e43d87dca9e'
# outfile = userID + '_profile.txt'

In [29]:
# Build user, paths, and count dictionary
# # traverse all users.. 
count = 0
user_song_count_dic = {}
with open('train_triplets.txt') as f: 
    for line in f:
        line = line.split('\n')[0]
        count += 1
        if count < 0:
            break
        else:
            userID = line.split('\t')[0]
            songID = line.split('\t')[1]
            playCount = line.split('\t')[2]
#             print(playCount)
            if songID in song_track_dic:
                trackID = song_track_dic[songID]
                if trackID in path_dict_song:
                    trackPath = path_dict_song[trackID]
                    if userID not in user_song_count_dic:
                        dic_playCount = {}
                        dic_playCount[trackPath] = playCount
                        user_song_count_dic[userID] = dic_playCount
                    else:
                        dic_playCount = user_song_count_dic[userID]
                        if trackPath not in dic_playCount:
                            dic_playCount[trackPath] = playCount
                            user_song_count_dic[userID] = dic_playCount

In [30]:
len(user_song_count_dic)

380904

In [7]:
# Don't execute this as data already saved
# traverse all users.. 
count = 0
user_song_dic = {}
with open('train_triplets.txt') as f: 
    for line in f:
        count += 1
        if count > 100000:
            break
        else:
            userID = line.split('\t')[0]
            songID = line.split('\t')[1]
            if songID in song_track_dic:
                trackID = song_track_dic[songID]
                if trackID in path_dict_song:
                    trackPath = path_dict_song[trackID]
                    if userID not in user_song_dic:
                        user_song_dic[userID] = [trackPath]
                    else:
                        track_list = user_song_dic[userID]
                        track_list.append(trackPath)
                        user_song_dic[userID] = track_list

In [40]:
rebuild_user_song_dic = {}
count = 0
with open('user_songPath.txt') as f:
    for line in f:
        count += 1
        if count < 0:
            break
        line = line.split('\n')[0]
        list_item = line.split('\t')
        userID = list_item[0]
        pathLst = [list_item[i] for i in range(1, len(list_item)-1)]
        rebuild_user_song_dic[userID] = pathLst
        

In [41]:
rebuild_user_song_dic['9215650e56636976a17e28c623d0830833f970f1']

['../MillionSongSubset/data/A/V/V/TRAVVZW12903CCEEE7.h5',
 '../MillionSongSubset/data/B/D/I/TRBDIOP128EF35F99C.h5',
 '../MillionSongSubset/data/A/N/D/TRANDEK128F4259EBC.h5',
 '../MillionSongSubset/data/A/A/K/TRAAKDG128F42A0ECB.h5',
 '../MillionSongSubset/data/A/S/P/TRASPAS128E078DFB3.h5',
 '../MillionSongSubset/data/A/F/W/TRAFWEV128F428D391.h5',
 '../MillionSongSubset/data/B/H/L/TRBHLDQ128F423EF10.h5',
 '../MillionSongSubset/data/B/D/Q/TRBDQVN128F425CF6D.h5',
 '../MillionSongSubset/data/B/I/F/TRBIFGK128F92F23A2.h5',
 '../MillionSongSubset/data/A/N/B/TRANBUW128F933C645.h5',
 '../MillionSongSubset/data/A/P/T/TRAPTOE128F4272000.h5']

In [56]:
len(rebuild_user_song_dic)
# filter top 10 users with path > 30
count = 0
top_10_users = []
for k,v in rebuild_user_song_dic.items():
    if count > 10:
        break
    if len(v) >= 30:
        count += 1
        top_10_users.append(k)

In [57]:
top_10_users

['4e73d9e058d2b1f2dba9c1fe4a8f416f9f58364f',
 'c1255748c06ee3f6440c51c439446886c7807095',
 'db6a78c78c9239aba33861dae7611a6893fb27d5',
 '738759001498928d8dcb054cd53a1a0cfc200d36',
 'ec6dfcf19485cb011e0b22637075037aae34cf26',
 '736083bd7ecd162effb7668cab6c281945762e85',
 '119b7c88d58d0c6eb051365c103da5caf817bea6']

In [66]:
# build FV for each of the 10 users and dump into csv
songH5File = None
for user in top_10_users:
    # initialize
    # ....
    lst_trackID = []
    lst_songID = []
    lst_artistFamiliarity = []
    lst_artistHottness = []
    lst_artistName = []
    lst_duration = []
    lst_dancebility = []
    lst_endOfFadeIn = []
    lst_energy = []
    lst_loudness = []
    lst_songHotness = []
    lst_tempo = []
    lst_songTitle = []
    # ....
    
    trackPaths = rebuild_user_song_dic[user]
    for track in trackPaths:
        # read the .h5 file
#         trackID = track.split('.h5')[0]
#         print(trackID)
        lst_trackID.append(track)
        
        songH5File = hdf5_getters.open_h5_file_read(track)
        
        songID = str(hdf5_getters.get_song_id(songH5File))
        lst_songID.append(songID)
        
        artistFamiliarity = hdf5_getters.get_artist_familiarity(songH5File)
        lst_artistFamiliarity.append(artistFamiliarity)
        
        artistHottness = hdf5_getters.get_artist_hotttnesss(songH5File)
        lst_artistHottness.append(artistHottness)
        
        artistName = hdf5_getters.get_artist_name(songH5File)
        lst_artistName.append(artistName)
        
        duration = hdf5_getters.get_duration(songH5File)
        lst_duration.append(duration)
        
        dancebility = hdf5_getters.get_danceability(songH5File)
        lst_dancebility.append(dancebility)
        
        endOfFadeIn = hdf5_getters.get_end_of_fade_in(songH5File)
        lst_endOfFadeIn.append(endOfFadeIn)
        
        energy = hdf5_getters.get_energy(songH5File)
        lst_energy.append(energy)
        
        loudness = hdf5_getters.get_loudness(songH5File)
        lst_loudness.append(loudness)
        
        songHotness = hdf5_getters.get_song_hotttnesss(songH5File)
        lst_songHotness.append(songHotness)
        
        tempo = hdf5_getters.get_tempo(songH5File)
        lst_tempo.append(tempo)
        
        songTitle = hdf5_getters.get_title(songH5File)
        lst_songTitle.append(songTitle)
        
        
        
    # construct dataframe for this user
    userFV = pd.DataFrame({'trackID': lst_trackID, 'songID': lst_songID, 'artistFamiliarity': lst_artistFamiliarity, 'artistHottness': lst_artistHottness,
                         'artistName': lst_artistName, 'duration': lst_duration, 'dancebility': lst_dancebility,
                         'endOfFadeIn': lst_endOfFadeIn, 'energy': lst_energy, 'loudness': lst_loudness, 'songHotness': lst_songHotness,
                         'tempo': lst_tempo, 'songTitle': lst_songTitle})
    outputFile = user + '_FV.csv'
    userFV.to_csv(outputFile)


In [26]:
# Don't execute this
# # dump user_song_dic
with open('user_songPath.txt', 'w') as ofile:
    output_str = ''
    for k,v in user_song_dic.items():
        output_str += k + '\t'
        for item in v:
            output_str += item + '\t'
        output_str += '\n'
    ofile.write(output_str)

In [39]:
# Don't execute, already saved
# # # dump user_song_dic
count = 0
with open('user_song_countPath.txt', 'w') as ofile:
    output_str = ''
    for k,v in user_song_count_dic.items():
        count += 1
        if count < 0:
            break
        output_str += k + '\t'
        for k_1,v_1 in v.items():
            output_str += k_1 + '\t' + v_1 + '\t'
        output_str += '\n'
    ofile.write(output_str)

In [36]:
len(user_song_count_dic)

380904

In [59]:
top_10_users

['4e73d9e058d2b1f2dba9c1fe4a8f416f9f58364f',
 'c1255748c06ee3f6440c51c439446886c7807095',
 'db6a78c78c9239aba33861dae7611a6893fb27d5',
 '738759001498928d8dcb054cd53a1a0cfc200d36',
 'ec6dfcf19485cb011e0b22637075037aae34cf26',
 '736083bd7ecd162effb7668cab6c281945762e85',
 '119b7c88d58d0c6eb051365c103da5caf817bea6']

In [76]:
eval_UserID = top_10_users[0]
trainX = pd.read_csv('4e73d9e058d2b1f2dba9c1fe4a8f416f9f58364f_FV.csv')

In [79]:
trainX.columns

Index(['Unnamed: 0', 'artistFamiliarity', 'artistHottness', 'artistName',
       'dancebility', 'duration', 'endOfFadeIn', 'energy', 'loudness',
       'songHotness', 'songID', 'songTitle', 'tempo', 'trackID'],
      dtype='object')

In [80]:
X_songCount = user_song_count_dic[eval_UserID]
trainX['playCount'] = trainX['trackID'].apply(lambda x: X_songCount[x])

In [102]:
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(trainX['playCount'])
trainX['normPlayCount'] = pd.DataFrame(x_scaled)

In [103]:
trainX

,Unnamed: 0,artistFamiliarity,artistHottness,artistName,dancebility,duration,endOfFadeIn,energy,loudness,songHotness,songID,songTitle,tempo,trackID,playCount,normPlayCount
0,0,0.806359,0.545789,b'Soundgarden',0.0,415.81669,0.000,0.0,-9.306,0.654068,b'SOAAEHR12A6D4FB060',b'Slaves & Bulldozers',131.272,../MillionSongSubset/data/A/Y/P/TRAYPFH128E079...,12,1.000000
1,1,0.609294,0.373060,b'Benabar',0.0,238.44526,4.905,0.0,-11.258,0.464924,b'SOADDVU12A8AE47512',b'Le Zoo De Vincennes',88.018,../MillionSongSubset/data/A/C/X/TRACXYU128F425...,1,0.000000
2,2,0.411905,0.354997,b'Charles Aznavour',0.0,147.77424,0.322,0.0,-10.819,0.345802,b'SOAKDLX12A67ADAFC5',"b""Tant Que L'on S'aimera""",100.618,../MillionSongSubset/data/A/V/B/TRAVBSQ128EF34...,1,0.000000
3,3,0.899935,0.675887,b'Radiohead',0.0,249.93914,0.183,0.0,-11.637,0.700049,b'SOAOPCG12AC468D27E',b'Everything In Its Right Place',122.831,../MillionSongSubset/data/B/H/K/TRBHKXX128F425...,1,0.000000
4,4,0.732661,0.458439,b'Del The Funky Homosapien',0.0,316.60363,0.235,0.0,-8.039,0.636287,b'SOBACYS12A8C13D6A4',b'Why You Wanna Get Funky...',83.646,../MillionSongSubset/data/A/W/B/TRAWBSZ128F427...,1,0.000000
5,5,0.858677,0.618219,b'Bad Religion',0.0,247.58812,0.000,0.0,-8.512,0.698494,b'SOBMNXE12A8C140F8A',b'Infected',121.524,../MillionSongSubset/data/A/X/R/TRAXRSK128F42B...,3,0.181818
6,6,0.804524,0.586433,b'Placebo',0.0,246.96118,0.340,0.0,-6.033,0.850283,b'SOCLAYI12A6310F138',b'Slave To The Wage',142.116,../MillionSongSubset/data/A/B/V/TRABVIU128E078...,1,0.000000
7,7,0.754918,0.507464,b'Rick Astley',0.0,211.69587,0.139,0.0,-7.750,0.864249,b'SOCWJDB12A58A776AF',b'Never Gonna Give You Up',113.359,../MillionSongSubset/data/A/X/L/TRAXLZU12903D0...,2,0.090909
8,8,0.804524,0.586433,b'Placebo',0.0,241.52771,0.000,0.0,-7.255,0.873702,b'SODWXQV12A6310F10D',b'English Summer Rain',108.264,../MillionSongSubset/data/A/A/U/TRAAUEO128E078...,1,0.000000
9,9,0.865022,0.916053,b'Coldplay',0.0,274.41587,0.322,0.0,-7.446,0.810264,b'SOEHTZE12A6310F0F2',b'One I Love',134.027,../MillionSongSubset/data/A/G/S/TRAGSGF128E078...,1,0.000000


In [121]:
# songID = np.array(trainX['songID'].apply(lambda x : x.split('b')[1].split('\'')[1]))

In [108]:
features = ['artistFamiliarity', 'artistHottness',
       'dancebility', 'duration', 'endOfFadeIn', 'energy', 'loudness',
       'tempo', 'songHotness', 'normPlayCount']
# handle missing values
for col in features:
    trainX[col].fillna((trainX[col].mean()), inplace=True)

newTrainX = trainX[features]
X_train = []
X_test = []
y_train = []
y_test = []
count = 0
for row in newTrainX.iterrows():
#     print(row)
    vec = []
    for col in range(len(row[1])-1):
        val = row[1][col]
#         print(val)
        if math.isnan(val):
            val = 0.0
        vec.append(val)
    if count <= 20:
        X_train.append(np.array(vec))
        y_train.append(row[1][-1])
        count += 1
    else:
        X_test.append(np.array(vec))
        y_test.append(row[1][-1])
        count += 1

y_train = np.array(y_train)
y_train = y_train.reshape(len(y_train), 1)
y_test = np.array(y_test)
y_test = y_test.reshape(len(y_test), 1)

X_train = np.array(X_train)
X_test = np.array(X_test)

In [109]:
y_train.shape

(21, 1)

In [110]:
X_train.shape

(21, 9)

In [111]:
y_train

array([[ 1.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.18181818],
       [ 0.        ],
       [ 0.09090909],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.18181818],
       [ 0.18181818],
       [ 0.        ],
       [ 0.36363636],
       [ 0.        ],
       [ 0.09090909]])

In [112]:
regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)
weights = regr.coef_
print('Weights learned', weights)

Weights learned [[ -1.63977761e-01  -2.12611207e-01  -2.67326911e-15   2.19157679e-03
   -6.65640817e-03   1.55342599e-16  -2.47446830e-03   9.48082292e-04
    3.58891993e-01]]


In [117]:
y_pred = regr.predict(X_test)
rmse = mean_squared_error(y_test, y_pred) 
print('rmse : ', rmse)

rmse :  0.0708429078045


In [118]:
y_test

array([[ 0.        ],
       [ 0.        ],
       [ 0.09090909],
       [ 0.36363636],
       [ 0.09090909],
       [ 0.45454545],
       [ 0.09090909],
       [ 0.18181818],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ]])

In [119]:
y_pred

array([[ 0.18612889],
       [ 0.05268768],
       [-0.03616764],
       [ 0.14765212],
       [-0.07235487],
       [-0.11781707],
       [ 0.04628885],
       [ 0.15920528],
       [ 0.20681976],
       [ 0.5257136 ],
       [ 0.05611463]])